<a href="https://colab.research.google.com/github/FabioLo464/Imers-o-Alura-AI-25/blob/main/Imers%C3%A3o_ALura_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [146]:
!pip install google-genai

In [147]:
#instalação do framework do google adk
!pip install -q google-adk

In [148]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


In [149]:
from google import genai

client = genai.Client()

In [150]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types #para criar conteudos
from datetime import date
import textwrap # Pra formatar melhor a saida de texto
from IPython.display import display, Markdown # Para exibir texto em formatado do colab
import requests #para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [151]:
for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-p

In [152]:
def call_agent(agent: Agent, message_text) -> str:
  # Cria um serviço de sessão em memória
  session_service = InMemorySessionService()

  # Cria uma nova sessão
  session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")

  # Criar um Runner para o agente
  runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
  # Cria o conteúdo da mensagem de entrada
  content = types.Content(role="user", parts=[types.Part(text=message_text)])

  final_response = ""
  #Itera assincronamente pelos eventos retornados durante a execução do agente
  for event in runner.run(user_id="user1", session_id="session1", new_message=content):
    if event.is_final_response():
     for part in event.content.parts:
       final_response += part.text
  return final_response


In [153]:
# Função auxiliar para exibir texto em formato em Markdown no colab
# Função auxiliar para exibir texto em formato em Markdown no colab
def to_markdown(text):
    text = text.replace('*', '  *')
    return Markdown(textwrap.indent(text, '>', predicate=lambda _: True))

In [219]:
def apresentar_quiz_gemini():
    try:
        prompt_geracao_quiz = "Gere uma pergunta de múltipla escolha sobre um tópico de cibersegurança. Inclua a pergunta, quatro opções de resposta (a, b, c, d) e indique qual é a resposta correta, juntamente com uma breve explicação."
        resposta_gemini = chat.send_message(prompt_geracao_quiz)
        print("\n--- Resposta Bruta do Gemini para o Quiz (Simplificado) ---\n")
        print(resposta_gemini.text)
        print("\n-----------------------------------------------------------\n")
    except Exception as e:
        print(f"Erro ao tentar obter resposta do Gemini para o quiz (simplificado): {e}")

In [213]:
# Agente 0 Boas Vindas
def agente_boas_vindas():
    boas_vindas_agente = Agent(
        name="agente_boas_vindas",
        model="gemini-2.0-flash",
        description='Agente para dar as boas-vindas ao usuário e engajá-lo.',
        instruction="""
        Você é um assistente amigável e especializado em cibersegurança. E o seu nome é ClaritySec. Sua tarefa é dar as boas-vindas ao usuário,
        informá-lo brevemente sobre sua capacidade de fornecer informações sobre diversos tópicos de cibersegurança
        e incentivá-lo a iniciar uma conversa.

        Comece com uma saudação calorosa. Em seguida, mencione alguns exemplos de tópicos sobre os quais você pode
        fornecer informações, como "phishing", "malware", "ransomware", "segurança de redes" ou "proteção de dados".

        Finalmente, faça uma pergunta aberta para encorajar o usuário a escolher um tópico. Por exemplo:
        'Olá! Bem-vindo ao seu especialista em cibersegurança. Posso te ajudar com informações sobre tópicos como phishing,
        malware, segurança de redes e muito mais. Sobre qual área de cibersegurança você gostaria de saber mais hoje?'
        """
    )
    return call_agent(boas_vindas_agente, "Iniciar conversa.")

In [214]:
# 1 - Agente buscador

def agente_buscador(topico):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description='Agente que busca informacoes no Google',
        tools=[google_search],
        instruction="""
       Você é um agente buscador de conteúdo especializado em cibersegurança.
       Sua tarefa é usar a ferramenta de busca do Google (google_search) para encontrar as últimas notícias e informações relevantes sobre o tópico de cibersegurança fornecido pelo usuário.
       Ao realizar a busca, priorize fontes confiáveis e reconhecidas na comunidade científica e na indústria de cibersegurança.
       Tente se concentrar nos 3 a 5 resultados mais relevantes e recentes. Na sua resposta, forneça um breve resumo das principais informações encontradas e, se disponível,
       mencione a fonte da informação e a data de publicação. Adapte sua busca para corresponder especificamente à pergunta do usuário, utilizando palavras-chave relevantes como "vulnerabilidade", "exploit", "ataque", "prevenção", "detecção", quando apropriado.
        """
    )

    entrada_do_agente_buscador = f"Tópico:{topico}"
    resultado_buscador = call_agent(buscador, entrada_do_agente_buscador)
    return resultado_buscador

In [215]:
# 2 - Agente planejador

def agente_planejador(topico, resultado_buscador):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        description='Agente que faz a triagem de conteúdos encontrados',
        instruction="""
        Você é um especialista em cibersegurança com a tarefa de analisar o conteúdo retornado pelo agente buscador sobre o tópico fornecido pelo usuário. Seu objetivo principal é selecionar as informações mais relevantes, precisas e úteis para o usuário.
        Ao fazer a triagem, priorize conteúdos que tenham relação direta com o tópico, que apresentem informações detalhadas e com profundidade adequada, e que provenham de fontes confiáveis e atuais.
        Se o conteúdo inicial for superficial ou não abordar completamente a pergunta do usuário, você pode utilizar a ferramenta de busca para buscar informações mais específicas ou para aprofundar aspectos particulares do tema.
        Identifique os principais pontos e conceitos presentes no conteúdo selecionado e organize-os de forma lógica para facilitar a compreensão e a redação por parte do próximo agente.
        Se houver diferentes perspectivas ou abordagens sobre o tópico, tente incluir conteúdo que as represente de forma equilibrada.
        """
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nConteúdo buscado: {resultado_buscador}"
    plano_gerado = call_agent(planejador, entrada_do_agente_planejador)
    return plano_gerado

In [223]:
# 3 - Agente Redador

def agente_redator(topico, plano_gerado):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        description='Agente redator de conteúdos encontrados',
        instruction="""
        Você é um agente redator especializado em cibersegurança. Sua tarefa é transformar o conteúdo selecionado e organizado pelo agente planejador em explicações claras e didáticas sobre o tópico fornecido pelo usuário.
        Ao escrever, considere que o público pode ter diferentes níveis de conhecimento em cibersegurança. Utilize uma linguagem simples e evite jargões técnicos sempre que possível. Caso seja necessário usar termos técnicos, forneça explicações concisas.
        Seu objetivo principal é garantir que o usuário compreenda o tópico da maneira mais eficaz possível. Para isso, escolha o formato de apresentação mais adequado:
        * Para explicações conceituais: Utilize parágrafos claros, exemplos e analogias.
        * Para dicas práticas: Use listas de itens ou instruções passo a passo.
        * Para definições: Forneça explicações concisas e diretas.
        * Se o tópico envolver etapas ou processos: Use listas numeradas ou fluxogramas (se possível em texto).
        * Se o tópico se beneficiar de comparações: Utilize tabelas ou listas lado a lado.
        Mantenha um tom informativo e acessível. Se relevante e útil, inclua links para fontes confiáveis ou materiais de leitura adicional que possam aprofundar o entendimento sobre o tópico.
        """
        )

    entrada_do_agente_redator = f"Tópico:{topico}\nPlano gerado: {plano_gerado}"
    conteudo_gerado = call_agent(redator, entrada_do_agente_redator)
    return conteudo_gerado

In [217]:
# 4 - Agente revisor

def agente_revisor(topico, conteudo_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        description='Agente revisor dos conteúdos gerador',
        tools=[google_search],
        instruction="""
        Você é um editor e revisor especializado em conteúdos de cibersegurança e segurança da informação. Sua tarefa é analisar o conteúdo gerado pelo agente redator, verificando sua clareza, precisão gramatical, relevância para o tópico fornecido e precisão factual.
        Além disso, verifique se a linguagem utilizada é adequada ao público-alvo, se a terminologia é consistente e se a formatação (se houver) contribui para a legibilidade. Certifique-se de que o conteúdo esteja alinhado com as informações e o plano fornecidos pelos agentes buscador e planejador.
        Se o conteúdo atender a todos os critérios de revisão, retorne a seguinte mensagem exata: "O conteúdo está pronto para ser publicado"
        Caso encontre problemas de clareza, gramática, relevância, precisão factual, adequação ao público, consistência ou formatação, destaque os trechos problemáticos e forneça sugestões concisas para correção ou melhoria. Se identificar informações faltantes ou imprecisas que necessitam de verificação, você pode usar a ferramenta de busca para esclarecimentos.
        """
    )

    entrada_do_agente_revisor = f"Tópico:{topico}\nConteúdo gerado:{conteudo_gerado}"
    conteudo_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return conteudo_revisado

In [224]:
print("Bem vindo ao seu especialista de Cibersegurança!")

# Chamar o agente de boas-vindas
mensagem_boas_vindas = agente_boas_vindas()
print("\n--- Resultado do Agente 0 (Boas Vindas) ---\n")
display(to_markdown(mensagem_boas_vindas))
print("------------------------------")
# obter topico do usuario
topico = input("Por favor, insira um tópico de pesquisa: ")

# inserir logica do sistema de agentes
if not topico:
    print("Por favor, insira um tópico de pesquisa.")
else:
    print(f"Ótimo, iremos informar você sobre o {topico} .")


    resultado_buscador = agente_buscador(topico)
    print("\n--- Resultado do Agente 1 (buscador) ---\n")
    display(to_markdown(resultado_buscador))
    print("------------------------------")

    plano_gerado = agente_planejador(topico, resultado_buscador)
    print("\n--- Resultado do Agente 2 (planejador) ---\n")
    display(to_markdown(plano_gerado))
    print("------------------------------")

    conteudo_gerado = agente_redator(topico, plano_gerado)
    print("\n--- Resultado do Agente 3 (redator) ---\n")
    display(to_markdown(conteudo_gerado))
    print("------------------------------")

    conteudo_revisado = agente_revisor(topico, conteudo_gerado)
    print("\n--- Resultado do Agente 4 (revisor) ---\n")
    display(to_markdown(conteudo_revisado))
    print("------------------------------")

#######
    perguntar_quiz = input("Gostaria de fazer um quiz sobre o tópico '{}'? (sim/não): ".format(topico)).lower()
    if perguntar_quiz == "sim":
        apresentar_quiz_gemini()

print("Foi um prazer ajudar! Se tiver mais dúvidas sobre cibersegurança, pode me chamar novamente.")

Bem vindo ao seu especialista de Cibersegurança!

--- Resultado do Agente 0 (Boas Vindas) ---



>Olá! Bem-vindo ao seu especialista em cibersegurança. Posso te ajudar com informações sobre tópicos como phishing, malware, segurança de redes e muito mais. Sobre qual área de cibersegurança você gostaria de saber mais hoje?

------------------------------
Por favor, insira um tópico de pesquisa: malware
Ótimo, iremos informar você sobre o malware .

--- Resultado do Agente 1 (buscador) ---



>Malware, abreviação de "software malicioso", refere-se a qualquer tipo de software projetado para causar danos a um dispositivo, sistema ou rede. O malware pode roubar informações confidenciais, danificar sistemas ou interromper operações.
>
>  *  *Tipos comuns de malware:  *  *
>  *     *  *Vírus:  *  * Programas maliciosos que se replicam infectando outros arquivos.
>  *     *  *Worms:  *  * Semelhantes aos vírus, mas se espalham para outros computadores através de uma rede.
>  *     *  *Cavalos de Tróia (Trojans):  *  * Disfarçam-se como software legítimo para obter acesso a um sistema e realizar ações maliciosas.
>  *     *  *Spyware:  *  * Monitora as atividades do usuário e coleta informações confidenciais sem o conhecimento do usuário.
>  *     *  *Adware:  *  * Exibe anúncios indesejados em um dispositivo infectado, muitas vezes prejudicando o desempenho do sistema.
>  *     *  *Ransomware:  *  * Criptografa os arquivos de um sistema e exige um resgate para descriptografá-los.
>  *     *  *Rootkits:  *  * Projetados para esconder sua presença e espiar o usuário, coletando dados.
>  *     *  *Malware sem arquivos:  *  * Utiliza programas legítimos para infectar um computador, tornando difícil a detecção.
>  *     *  *Keyloggers:  *  * Registram as teclas digitadas pelo usuário, capturando informações como senhas.
>  *     *  *Bots e botnets:  *  * Um bot é um computador infectado com malware para ser controlado remotamente e usado para lançar ataques em larga escala.
>  *     *  *Malvertising:  *  * Distribui conteúdo malicioso através de anúncios online.
>  *     *  *Bomba lógica:  *  * Código malicioso que é ativado quando uma condição específica é atendida.
>  *     *  *Scareware:  *  * Tenta enganar o usuário, simulando problemas no sistema para induzi-lo a adquirir software de segurança inútil.
>
>  *  *Prevenção contra malware:  *  *
>  *   Mantenha o software e o sistema operacional atualizados.
>  *   Instale e mantenha um software antivírus atualizado.
>  *   Tenha cuidado ao clicar em links ou baixar arquivos de fontes desconhecidas.
>  *   Use contas de usuário limitadas em vez de contas de administrador.
>  *   Faça backup de seus dados regularmente.
>  *   Instale uma firewall.
>
>  *  *Análise de malware:  *  *
>A análise de malware é um componente crítico na defesa contra ameaças cibernéticas. Existem várias técnicas avançadas utilizadas na análise de malware, incluindo:
>
>  *     *  *Análise Dinâmica com Sandbox:  *  * Executa o malware em um ambiente controlado e isolado para observar seu comportamento.
>  *     *  *Engenharia Reversa:  *  * Reverte o código binário do malware para entender seu funcionamento detalhado.
>  *     *  *Análise de Tráfego de Rede:  *  * Monitora o tráfego de rede gerado pelo malware para identificar comunicações com servidores de comando e controle.
>  *     *  *Análise Comportamental:  *  * Observa o comportamento do malware em relação ao sistema, analisando chamadas de API e modificações de arquivos.
>  *     *  *Análise de Código Ofuscado:  *  * Desofuscar o código para expor suas funcionalidades.
>
>  *  *Notícias Recentes:  *  *
>  *   Especialistas encontraram um vírus no iPhone que lê imagens do celular.
>  *   Um novo malware para Android, "Crocodilus", pode assumir o controle de celulares para roubar criptomoedas.
>  *   Golpes de "drainer" de criptomoedas estão sendo vendidos como malware fácil de usar.
>  *   O grupo de hackers russo ColdRiver está usando um novo malware chamado LostKeys para roubar arquivos de governos ocidentais, jornalistas e ONGs.
>  *   Um novo malware tem como alvo sites WordPress, disfarçando-se como uma ferramenta de segurança.
>  *   Um pacote malicioso do PyPi tem como alvo desenvolvedores do Discord com um trojan de acesso remoto desde 2022.
>  *   Um ataque à cadeia de suprimentos comprometeu um pacote npm, injetando código ofuscado que ativa um trojan de acesso remoto.
>  *   Um plugin WordPress chamado OttoKit, com mais de 100 mil instalações, foi atingido por malware.
>  *   O malware DarkGate explorou recentemente uma vulnerabilidade corrigida pela Microsoft em um ataque de dia zero.
>  *   O Trojan Android Banking PixPirate está usando uma nova tática de evasão para atingir usuários brasileiros.
>  *   Mais de 38 mil subdomínios FreeDrain foram encontrados explorando SEO para roubar frases iniciais de carteira de criptomoedas.
>  *   Sites falsificados do Zoom, Skype e Google Meet estão distribuindo malware.
>  *   Hackers exploram servidores mal configurados de YARN, Docker, Confluence, Redis para mineração de cripto.
>
>A CISA (Cybersecurity and Infrastructure Security Agency) lançou um sistema de análise de malware de última geração para uso público, permitindo que organizações e indivíduos analisem automaticamente arquivos e URLs potencialmente maliciosos.


------------------------------

--- Resultado do Agente 2 (planejador) ---



...

------------------------------

--- Resultado do Agente 3 (redator) ---



>## Malware: Entenda o que é, como funciona e como se proteger
>
>Malware é um termo abrangente para qualquer tipo de software criado com o objetivo de causar danos a um computador, rede ou sistema. Imagine um vírus que ataca seu corpo, mas no mundo digital. Esses programas maliciosos podem roubar informações, danificar arquivos, interromper operações e muito mais.
>
>### Tipos Comuns de Malware
>
>Existem diversos tipos de malware, cada um com suas próprias características e formas de atuação:
>
>  *     *  *Vírus:  *  * Assim como um vírus biológico, ele se replica e se espalha infectando outros arquivos. Precisa da ação do usuário para se propagar.
>  *     *  *Worms (Vermes):  *  * Semelhantes aos vírus, mas se espalham automaticamente através de redes, explorando vulnerabilidades.
>  *     *  *Trojans (Cavalos de Troia):  *  * Disfarçam-se como programas legítimos para enganar o usuário e abrir portas para outras ameaças.
>  *     *  *Spyware (Programas Espiões):  *  * Monitoram suas atividades online, coletam informações pessoais e as enviam para terceiros.
>  *     *  *Adware (Programas de Publicidade):  *  * Exibem anúncios indesejados e podem redirecionar você para sites maliciosos.
>  *     *  *Ransomware (Software de Resgate):  *  * Criptografam seus arquivos e exigem um resgate para devolvê-los.
>  *     *  *Rootkits:  *  * Dão acesso não autorizado ao sistema, permitindo que outros malwares se escondam e atuem sem serem detectados.
>  *     *  *Malware sem arquivos:  *  * Vivem na memória do computador, tornando-se difíceis de detectar e remover.
>  *     *  *Keyloggers:  *  * Registram tudo o que você digita, incluindo senhas e informações confidenciais.
>  *     *  *Bots e botnets:  *  * Um bot é um programa que infecta um computador e o controla remotamente. Vários computadores infectados formam uma botnet, usada para ataques em larga escala.
>  *     *  *Malvertising:  *  * Malware disseminado através de anúncios online aparentemente inofensivos.
>  *     *  *Bomba lógica:  *  * Código malicioso que é ativado quando uma condição específica é atendida, como uma data ou ação do usuário.
>  *     *  *Scareware:  *  * Assusta o usuário com falsos alertas de segurança, induzindo-o a comprar um software inútil ou malicioso.
>
>### Como se proteger de malware
>
>A prevenção é a melhor defesa contra malware. Aqui estão algumas dicas importantes:
>
>  *     *  *Mantenha seu software e sistema operacional atualizados:  *  * As atualizações corrigem falhas de segurança que os malwares podem explorar.
>  *     *  *Use um antivírus:  *  * Um bom antivírus pode detectar e remover malware antes que ele cause danos.
>  *     *  *Tenha cuidado com links e downloads:  *  * Não clique em links suspeitos ou baixe arquivos de fontes não confiáveis.
>  *     *  *Use contas de usuário limitadas:  *  * Isso impede que o malware faça alterações drásticas no sistema.
>  *     *  *Faça backup de seus dados:  *  * Se você for infectado por ransomware, poderá restaurar seus arquivos a partir do backup.
>  *     *  *Instale um firewall:  *  * Um firewall ajuda a bloquear o acesso não autorizado ao seu computador.
>
>### Análise de Malware: Detetives Digitais em Ação
>
>Quando um novo malware surge, especialistas em segurança cibernética precisam analisá-lo para entender como funciona e como combatê-lo. Eles usam diversas técnicas:
>
>  *     *  *Análise Dinâmica com Sandbox:  *  * Executam o malware em um ambiente isolado (sandbox) para observar seu comportamento sem risco para o sistema real.
>  *     *  *Engenharia Reversa:  *  * Desmontam o código do malware para entender sua estrutura e funcionalidade.
>  *     *  *Análise de Tráfego de Rede:  *  * Monitoram a comunicação do malware com outros computadores para identificar servidores de comando e controle.
>  *     *  *Análise Comportamental:  *  * Observam as ações do malware no sistema, como criação de arquivos, modificação de registros e comunicação com outros processos.
>  *     *  *Análise de Código Ofuscado:  *  * Decifram o código do malware para revelar sua verdadeira função, mesmo que esteja disfarçado.
>
>### Malware em notícias
>
>O mundo do malware está em constante evolução. Ataques recentes têm como alvo desde dispositivos móveis até grandes empresas e governos. É importante estar ciente das últimas ameaças para se proteger adequadamente.
>
>  *  *Recursos:  *  *
>
>  *   A CISA (Cybersecurity and Infrastructure Security Agency) oferece um sistema de análise de malware de última geração para uso público, ajudando a fortalecer a defesa cibernética.

------------------------------

--- Resultado do Agente 4 (revisor) ---



>O conteúdo está pronto para ser publicado

------------------------------
Gostaria de fazer um quiz sobre o tópico 'malware'? (sim/não): nao
Foi um prazer ajudar! Se tiver mais dúvidas sobre cibersegurança, pode me chamar novamente.


In [212]:
prompt = input("Usuário: ")

while prompt.lower() != "tchau":
    try:
        resposta = chat.send_message(prompt)
        print("Chatbot:", resposta.text)
    except Exception as e:
        print(f"Chatbot: Desculpe, ocorreu um erro ao processar sua mensagem: {e}")
        print("Por favor, tente novamente ou diga 'tchau' para encerrar.")
    prompt = input("Usuário: ")

print("Foi um prazer ajudar! Se tiver mais dúvidas sobre cibersegurança, pode me chamar novamente.")

Usuário: tchau
Foi um prazer ajudar! Se tiver mais dúvidas sobre cibersegurança, pode me chamar novamente.


In [ ]:
modelo = "gemini-2.0-flash"

resposta = client.models.generate_content(model=modelo,
                                          contents="o que é malwarel?")

In [ ]:
chat = client.chats.create(model=modelo)
resposta = chat.send_message("Olá, bem vindo ao assistente de Cibersegurança")

resposta.text

'Olá! É um prazer estar aqui para ajudar com suas dúvidas e necessidades em cibersegurança. \n\nEm que posso ser útil hoje? Seja específico sobre o que você precisa para que eu possa fornecer a melhor informação possível. Por exemplo:\n\n*   Você quer saber mais sobre um ataque cibernético específico?\n*   Precisa de ajuda para proteger seus dispositivos?\n*   Gostaria de dicas de segurança para navegar na internet?\n*   Quer entender melhor alguma lei ou regulamentação sobre proteção de dados?\n\nEstou pronto para te auxiliar!\n'

In [ ]:
#Perguntar ao Gemni uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

#perguntar pro modelo quando é a proxima sobre algo atual
resposta = client.models.generate_content(
    model=modelo,
    contents="Quando foi o último ataque de Ransomware que mais pagou resgaste em 2025?",
    config={"tools": [{"google_search":{}}]}
)
#print da resposta
display(Markdown(f"Response:\n {resposta.text}"))

Response:
 Para responder a sua pergunta sobre o ataque de ransomware mais caro de 2025, preciso realizar uma pesquisa atualizada.

Com base nas informações mais recentes até o primeiro trimestre de 2025, não há um ataque de ransomware único que se destaque como o mais caro em termos de resgate pago. No entanto, posso fornecer alguns dados relevantes e tendências observadas nesse período:

*   **Crescimento dos ataques:** Os ataques de ransomware aumentaram significativamente no primeiro trimestre de 2025, com um aumento de 126% em relação ao ano anterior.
*   **Resgates:** A média dos pagamentos de resgate também aumentou, ultrapassando 1,3 milhão de libras (aproximadamente US$ 1,6 milhão).
*   **Grupos notórios:** Grupos como Cl0p, LockBit 3.0 e BlackCat (ALPHV) estão entre os mais ativos e perigosos, com o LockBit sendo responsável por mais de 1.400 ataques apenas no primeiro trimestre.
*   **Alvos:** Os ataques estão se tornando mais sofisticados e direcionados, afetando setores como saúde, manufatura, educação e governo. O Brasil também tem sido um alvo frequente, registrando um número elevado de ataques.
*   **Tendências:** Há uma crescente profissionalização dos criminosos, com o uso de modelos de "Ransomware como Serviço" (RaaS), dupla extorsão (criptografia e ameaça de vazamento de dados) e táticas de engenharia social.

Embora não seja possível apontar o ataque mais caro especificamente, os relatórios indicam que os valores dos resgates estão aumentando e que os ataques estão se tornando mais frequentes e impactantes.


In [ ]:
from google.genai import types

chat_config = types.GenerateContentConfig(system_instruction= "Você é um assistente de Cibersegurança que consegue responder de forma sucinta e didática")

chat= client.chats.create(model=modelo,
                          config=chat_config)


In [ ]:
prompt_usuario = input("Usuário: ")
prompt_gemini = f"{prompt_usuario}. Forneça uma explicação detalhada, incluindo dicas práticas, exemplos e, se possível, links para recursos confiáveis sobre este tópico."
resposta = chat.send_message(prompt_gemini)
print("Chatbot:", resposta.text)

Usuário: forneça links sobre malware
Chatbot: Com prazer! Vamos mergulhar no mundo do malware.

**O que é Malware?**

Malware é um termo genérico para software malicioso projetado para causar danos a um computador, rede ou sistema. O objetivo pode variar desde roubar dados, espionar atividades online, danificar arquivos, até mesmo assumir o controle do sistema.

**Tipos Comuns de Malware:**

*   **Vírus:** Se propagam infectando arquivos e se replicando. Precisam de um hospedeiro para se espalhar.
*   **Worms:** Se replicam e se espalham automaticamente através de redes, sem precisar de um hospedeiro.
*   **Trojans (Cavalos de Troia):** Disfarçam-se como software legítimo, mas executam atividades maliciosas em segundo plano.
*   **Ransomware:** Criptografam os arquivos da vítima e exigem um resgate para descriptografá-los.
*   **Spyware:** Coletam informações sobre a vítima sem o seu conhecimento e as enviam para terceiros.
*   **Adware:** Exibem anúncios indesejados e podem coletar da

In [ ]:
chat.get_history()

[UserContent(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='o que é o misp?')], role='user'),
 Content(parts=[Part(video_metadata=None, thought=None, inline_data=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, text='O MISP (Malware Information Sharing Platform) é uma plataforma de código aberto para **compartilhamento de informações sobre ameaças cibernéticas**, incluindo indicadores de comprometimento (IoCs), análise de malware e informações sobre incidentes de segurança.\n\n**Em termos simples:**\n\n*   É como uma rede social para profissionais de segurança cibernética.\n*   Permite que as organizações compartilhem informações sobre ameaças de forma estruturada e automatizada.\n*   Ajuda a melhorar a detecção, prevenção e resposta a incidentes de segurança.\n\n**Benefícios:**\n\n*   **